In [1]:
from utils_1.goenv import GoEnv,_format_state, create_env_copy
from utils_1.fnet import NeuralTrainer, AlphaNeural
import numpy as np
import goSim
from tqdm.auto import tqdm

In [2]:
class AlphaGoPlayer():
    def __init__(self, init_state, seed, player_color):
        # setup parameters
        self.init_state = init_state
        self.seed = seed
        self.player_color = player_color
        
        # init environment
        if player_color==1:
            self.env = GoEnv(player_color="black", board_size=13)
        else:
            self.env = GoEnv(player_color="white", board_size=13)
        self.env.reset()
        
        # load neural network
        self.model = NeuralTrainer(res_blocks=10, board_size=13)
#         self.model.load_model("utils_1/networks/testing1.model")
        
    def get_action(self, cur_state, opponent_action):
        if opponent_action>=0:
            self.env.step(opponent_action)
            print('-------------------------------------')
            print('opponent_action: ' + str(opponent_action))
            print('-------------------------------------')
        
        policy, value = self.model.predict(self.env.get_history())
        legal_moves = self.env.get_legal_moves()
        policy = policy*legal_moves
        action = np.argmax(policy)
        self.env.step(action)
        
        print('-------------------------------------')
        print('player_action: ' + str(action))
        print('-------------------------------------')
        self.env.print_board()
        
        return action

In [3]:
class MinMax:
    def __init__(self, board_size, player, max_depth=3):
        self.board_size=board_size
        self.max_depth = max_depth
        self.player = player
        
    def alpha_beta_search(self, board):
        # if player is black then do minimization at root
        if self.player==1:
            action_value = [np.inf]*(self.board_size**2+1)
            action_space = np.nonzero(board.get_legal_moves())[0]
            for action in tqdm(action_space):
                child = create_env_copy(board)
                try:
                    child.step(action)
                except (Exception):
                    print(action_space)
                    print(action)

                action_value[action] = self.max_search(child, -np.inf, np.inf,1)
            return np.argmin(action_value)
        
        # if player is white then do maximization at root
        else:
            action_value = [-np.inf]*(self.board_size**2+1)
            action_space = np.nonzero(board.get_legal_moves())[0]
            for action in tqdm(action_space):
                child = create_env_copy(board)
                child.step(action)
                action_value[action] = self.max_search(child, -np.inf, np.inf,1)
            return np.argmax(action_value)
    
    def max_search(self, board, alpha, beta, depth):
        if board.isComplete():
            return board.get_winner()*1000
        if depth==self.max_depth:
            return board.curr_score()
        
        v = -np.inf
        
        action_space = np.nonzero(board.get_legal_moves())[0]
        
        for action in action_space:
            child = create_env_copy(board)
            try:
                child.step(action)
            except (Exception):
                print(action_space)
                print(action)
            
            v = max(v,self.min_search(child,alpha,beta,depth+1))
            if v>= beta:
                return v
            alpha = max(alpha,v)
        return v
    
    def min_search(self, board, alpha, beta, depth):
        if board.isComplete():
            return board.get_winner()*1000
        if depth==self.max_depth:
            return board.curr_score()
        
        v = np.inf
        action_space = np.nonzero(board.get_legal_moves())[0]
        
        for action in action_space:
            child = create_env_copy(board)
            child.step(action)
            
            v = min(v,self.max_search(child,alpha,beta, depth+1))
            if v<= alpha:
                return v
            beta = min(beta,v)
        return v

In [4]:
class AlphaGoPlayer_MinMax():
    def __init__(self, init_state, seed, player_color):
        # setup parameters
        self.init_state = init_state
        self.seed = seed
        self.player_color = player_color
        
        self.minmax = MinMax(board_size=13, player=player_color, max_depth=2)
        
        # init environment
        if player_color==1:
            self.env = GoEnv(player_color="black", board_size=13)
        else:
            self.env = GoEnv(player_color="white", board_size=13)
        self.env.reset()
        
    def get_action(self, cur_state, opponent_action):
        if opponent_action>=0:
            self.env.step(opponent_action)
        
        action = self.minmax.alpha_beta_search(self.env)
        self.env.step(action)
        
        return action

In [5]:
env = goSim.GoEnv(player_color="black", observation_type='image3c', illegal_move_mode="raise", board_size=13, komi=7.5)

obs_t = env.reset()
opp_action=-1

In [6]:
a = AlphaGoPlayer_MinMax(obs_t.copy(), 1, 1)

In [7]:
env.set_player_color(1)
a_t = a.get_action(obs_t.copy(), opp_action)
obs_t, a_t, r_t, done, info, cur_score = env.step(a_t)


0


In [8]:
env.set_player_color(2)
opp_action = np.random.randint(0,170)
obs_t, a_t, r_t, done, info, cur_score = env.step(opp_action)

11


In [9]:
env.set_player_color(1)
a_t = a.get_action(obs_t.copy(), opp_action)
obs_t, a_t, r_t, done, info, cur_score = env.step(a_t)


24


In [10]:
env.set_player_color(2)
opp_action = np.random.randint(0,170)
obs_t, a_t, r_t, done, info, cur_score = env.step(opp_action)

124


In [11]:
env.set_player_color(1)
a_t = a.get_action(obs_t.copy(), opp_action)
obs_t, a_t, r_t, done, info, cur_score = env.step(a_t)


1


In [12]:
env = goSim.GoEnv(player_color="black", observation_type='image3c', illegal_move_mode="raise", board_size=13, komi=7.5)
obs_t = env.reset()
opp_action=-1
a = AlphaGoPlayer(obs_t.copy(), 2, 1)


env.set_player_color(1)
opp_action = np.random.randint(0,170)
obs_t, a_t, r_t, done, info, cur_score = env.step(opp_action)

149


In [13]:
env.set_player_color(2)
a_t = a.get_action(obs_t.copy(), opp_action)
obs_t, a_t, r_t, done, info, cur_score = env.step(a_t)

-------------------------------------
opponent_action: 149
-------------------------------------
-------------------------------------
player_action: 1
-------------------------------------
. O . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . X . . . . . . 
. . . . . . . . . . . . . 
1


In [14]:
env.set_player_color(1)
opp_action = np.random.randint(0,170)
obs_t, a_t, r_t, done, info, cur_score = env.step(opp_action)

env.set_player_color(2)
a_t = a.get_action(obs_t.copy(), opp_action)
obs_t, a_t, r_t, done, info, cur_score = env.step(a_t)


102
-------------------------------------
opponent_action: 102
-------------------------------------
-------------------------------------
player_action: 123
-------------------------------------
. O . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . X . 
. . . . . . . . . . . . . 
. . . . . . O . . . . . . 
. . . . . . . . . . . . . 
. . . . . . X . . . . . . 
. . . . . . . . . . . . . 
123


In [16]:
env_mank = GoEnv("black", board_size=13)
s = env_mank.reset();

In [18]:
a1 = AlphaGoPlayer(s,1,1)

In [19]:
a2 = AlphaGoPlayer_MinMax(s, 1, 2)

In [20]:
for i in range(200):
    s = env_mank.get_history()
    policy, value = fnet.predict(s)
    policy = env_mank.get_legal_moves()*policy
    action = np.argmax(policy)
    env_mank.step(action)
    if (env_mank.isComplete()):
        print(env_mank.get_winner())
        break
    print(env_mank.curr_score())    
    legal_moves = np.nonzero(env_mank.get_legal_moves())[0]
    opp_action = a.get_action(s,action)
    env_mank.step(opp_action)
    print(env_mank.curr_score())
    env_mank.print_board()
    if (env_mank.isComplete()):
        print(env_mank.get_winner())
        break

6.5



7.5
O . . . . . . . . . . . . 
. . . . . . . . X . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
6.5



7.5
O O . . . . . . . . . . . 
. . . . . . . . X . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . X . . . . . . . . . 
. . . . . . . . . . . . . 
6.5



7.5
O O O . . . . . . . . . . 
. . . . . . . . X . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. X . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . X . . . . . . . . . 
. . . . . . . . . . . . . 
6.5



7.5
O O O O . . . . . . . . . 
. . . . . . . . X . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. X . . . . . . . . . . X 
. . . . . . . . . . . . . 
. . . . . . . . . . . . . 
. . . X . . . . . . . . . 
. . . . . . . . . . . . . 
6.5


...


-65.5
O O O O O O O O X O O O O 
O X X O O . X X X . X . . 
. . . . . X X . X . . . . 
. . . X . . X X X X . . . 
. . . . . . . . . X . X X 
X . X . . . X . X X . X . 
X . . . . . . . X X X X X 
X . . . . X . X X X X X X 
X X . . . . . X X X X X X 
X X X X . X X O O O X X X 
X X X X X X X O X X X X X 
X X X X X X X . O X X X X 
X X X X X X X X X X X X X 
-65.5



-64.5
O O O O O O O O X O O O O 
O X X O O O X X X . X . . 
. . . . . X X . X . . . . 
. . . X . . X X X X . . . 
. . . . . . . . . X . X X 
X . X . . . X . X X . X . 
X . . . . . . . X X X X X 
X . . . . X . X X X X X X 
X X . . . . . X X X X X X 
X X X X . X X O O O X X X 
X X X X X X X O X X X X X 
X X X X X X X . O X X X X 
X X X X X X X X X X X X X 
-65.5



-64.5
O O O O O O O O X O O O O 
O X X O O O X X X O X . . 
X . . . . X X . X . . . . 
. . . X . . X X X X . . . 
. . . . . . . . . X . X X 
X . X . . . X . X X . X . 
X . . . . . . . X X X X X 
X . . . . X . X X X X X X 
X X . . . . . X X X X X X 
X X X X . X X O O O X X X 
X X X X X X X O X X X X X 
X X X X X X X . O X X X X 
X X X X X X X X X X X X X 
-64.5



-63.5
O O O O O O O O X O O O O 
O X X O O O X X X O X O . 
X . . . . X X . X . . . . 
. . . X . . X X X X . . . 
. . . . . . . . . X . X X 
X . X . . . X . X X . X . 
X . . . . . . . X X X X X 
X . . . . X . X X X X X X 
X X . . . . . X X X X X X 
X X X X . X X O O O X X X 
X X X X X X X O X X X X X 
X X X X X X X . O X X X X 
X X X X X X X X X X X X X 
-63.5



-62.5
O O O O O O O O X O O O O 
O X X O O O X X X O X O O 
X . . . . X X . X . . . . 
. . . X . . X X X X . . . 
. . . . . . . . . X . X X 
X . X . . . X . X X . X . 
X . . . . . . . X X X X X 
X . . . . X . X X X X X X 
X X . . . . . X X X X X X 
X X X X . X X O O O X X X 
X X X X X X X O X X X X X 
X X X X X X X . O X X X X 
X X X X X X X X X X X X X 


In [15]:
a.get_action

5.5